In [1]:
import numpy as np
import pandas as pd
import optuna
from scipy.sparse import csr_matrix
from weighting_strategies import (
    bm25_weight, tfidf_weight, normalized_weight,
    log_weight, confidence_weight, power_weight,
    pmi_weight, robust_user_centric_weight, sigmoid_propensity_weight, power_lift_weight, robust_user_centric_weight_v2
)
from implicit.nearest_neighbours import CosineRecommender
from implicit.evaluation import train_test_split, precision_at_k, ndcg_at_k

import cornac


In [2]:
import sys
import os

# Add the parent directory to sys.path to resolve imports from sibling directories
sys.path.append(os.path.abspath(".."))

from utils.sparse import transform_dataframe_to_sparse

In [3]:
movielens_df = (
    pd.DataFrame(
        data=cornac.datasets.movielens.load_feedback(variant="100K"),
        columns=['user_id', 'item_id', 'target']
    )
    .loc[:, ['user_id', 'item_id', 'target']]
    .dropna()
)
movielens_df['user_id'].nunique(), movielens_df['item_id'].nunique(), movielens_df.shape[0]

(943, 1682, 100000)

In [4]:
user_item_matrix, user_mapping, item_mapping = transform_dataframe_to_sparse(
    movielens_df, row_field='user_id', col_field='item_id', data_field='target'
)


train_val_mat, test_mat = train_test_split(user_item_matrix, train_percentage=0.9, random_state=42)
train_mat, val_mat = train_test_split(train_val_mat, train_percentage=0.9, random_state=42)

print(f"Train Shape: {train_mat.shape}, Val Shape: {val_mat.shape}, Test Shape: {test_mat.shape}")

Train Shape: (943, 1682), Val Shape: (943, 1682), Test Shape: (943, 1682)


In [ ]:
results_folder = "results/movielens_100k_knn"
results_filename = "movielens_100k_knn_results.csv"

import time

def run_hyperparameter_optimization(
    train_mat: csr_matrix,
    val_mat: csr_matrix,
    train_val_mat: csr_matrix,
    test_mat: csr_matrix,
    weighting_strategy: str,
    algorithm: str,
    n_trials: int = 20,
    output_dir: str = None,
) -> pd.DataFrame:
    results = []
    algorithms = {
        "KNN_k=20": lambda: CosineRecommender(K=20),
        "KNN_k=100": lambda: CosineRecommender(K=100),
    }
    strategies = [
        "no_weighting",
        "bm25",
        "tfidf",
        "log", 
        "confidence",
        "power",
        "normalized",
        "pmi",
        "robust_user_centric",
        "robust_user_centric_weight_v2",
        "sigmoid_propensity",
        "power_lift"
    ]
    if weighting_strategy not in strategies:
        raise ValueError(f"Weighting strategy '{weighting_strategy}' is not recognized.")
    strategy = weighting_strategy

    if algorithm not in algorithms:
        raise ValueError(f"Algorithm '{algorithm}' is not recognized.")
    algo_name = algorithm
    AlgoFactory = algorithms[algorithm]

    print(f"Running optimization for {algo_name} with {strategy}...")

    def get_weighted_matrix(matrix, params):
        weighted = matrix.copy()
        if strategy == "bm25":
            weighted = bm25_weight(weighted, K1=params.get("bm25_k1"), B=params.get("bm25_b"))
        elif strategy == "confidence":
            weighted = confidence_weight(weighted, alpha=params.get("conf_alpha"))
        elif strategy == "power":
            weighted = power_weight(weighted, p=params.get("power_p"))
        elif strategy == "tfidf":
            weighted = tfidf_weight(weighted)
        elif strategy == "log":
            weighted = log_weight(weighted)
        elif strategy == "normalized":
            weighted = normalized_weight(weighted)
        elif strategy == "pmi":
            weighted = pmi_weight(weighted)
        elif strategy == "robust_user_centric":
            weighted = robust_user_centric_weight(weighted, scale_factor=params.get("scale_factor"))
        elif strategy == "sigmoid_propensity":
            weighted = sigmoid_propensity_weight(weighted, p=params.get("p"), beta=params.get("beta"))
        elif strategy == "power_lift":
            weighted = power_lift_weight(weighted, p=params.get("p"))
        elif strategy == "robust_user_centric_weight_v2":
            weighted = robust_user_centric_weight_v2(weighted, lower_q=params.get("lower_q"), upper_q=params.get("upper_q"))
        return weighted

    def objective(trial):
        params = {}
        # Suggest weighting strategy parameters
        if strategy == "bm25":
            params["bm25_k1"] = trial.suggest_float("bm25_k1", 0.1, 1000)
            params["bm25_b"] = trial.suggest_float("bm25_b", 0.0, 1.0)
        elif strategy == "confidence":
            params["conf_alpha"] = trial.suggest_float("conf_alpha", 1.0, 150.0)
        elif strategy == "power":
            params["power_p"] = trial.suggest_float("power_p", 0.1, 1.5)
        elif strategy == "robust_user_centric":
            params["scale_factor"] = trial.suggest_float("scale_factor", 0.1, 10.0)
        elif strategy == "robust_user_centric_weight_v2":
            params["lower_q"] = trial.suggest_float("lower_q", 5.0, 45.0)
            params["upper_q"] = trial.suggest_float("upper_q", 55.0, 95.0)
        elif strategy == "sigmoid_propensity":
            params["p"] = trial.suggest_float("p", 0.1, 5.0)
            params["beta"] = trial.suggest_float("beta", 0.0, 1.0)
        elif strategy == "power_lift":
            params["p"] = trial.suggest_float("p", 0.1, 1.5)
        weighted_train = get_weighted_matrix(train_mat, params)

        # Train Model
        model = AlgoFactory()
        model.fit(weighted_train, show_progress=False)

        # Evaluate on Validation Set
        return ndcg_at_k(model, train_mat, val_mat, K=20, show_progress=False)

    # Optimize only if strategy has parameters
    current_trials = n_trials if strategy in ["bm25", "confidence", "power", "robust_user_centric", "robust_user_centric_weight_v2", "sigmoid_propensity", "power_lift"] else 1
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=current_trials, n_jobs=-1)

    # --- Final Retraining & Testing ---
    # Use best params to weight the full train_val matrix
    best_params = study.best_params
    weighted_train_val = get_weighted_matrix(train_val_mat, best_params)

    # Train Final Model
    final_model = AlgoFactory()
    
    start_time = time.time()
    final_model.fit(weighted_train_val, show_progress=False)
    end_time = time.time()
    
    # Evaluate on Test Set
    test_ndcg_10 = ndcg_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    test_precision_10 = precision_at_k(final_model, train_val_mat, test_mat, K=10, show_progress=False)
    test_ndcg_20 = ndcg_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)
    test_precision_20 = precision_at_k(final_model, train_val_mat, test_mat, K=20, show_progress=False)

    results.append({
        "Algorithm": algo_name,
        "Strategy": strategy,
        "Number of Optimization Trials": current_trials,
        "Best Val NDCG@20": study.best_value,
        "Test NDCG@10": test_ndcg_10,
        "Test NDCG@20": test_ndcg_20,
        "Test Precision@10": test_precision_10,
        "Test Precision@20": test_precision_20,
        "Final Train Time (s)": end_time - start_time,
        "Best Params": best_params
    })

    if output_dir:
        output_path = os.path.join(output_dir, f"{algo_name}_{strategy}_results.csv")
        pd.DataFrame(results).to_csv(output_path, index=False)
    return pd.DataFrame(results)

In [6]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [7]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:34,285] A new study created in memory with name: no-name-fcd06269-28bb-4b59-aeb0-11c8cd63c3f2
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005834102630615234 seconds
  warnings.warn(
[I 2026-02-08 12:33:34,404] Trial 0 finished with value: 0.23373554820119585 and parameters: {}. Best is trial 0 with value: 0.23373554820119585.


Running optimization for KNN_k=20 with no_weighting...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005936622619628906 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,no_weighting,1,0.233736,0.267649,0.283587,0.287811,0.319398,0.021264,{}


In [8]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:34,735] A new study created in memory with name: no-name-bef18d5c-b9c7-4b9a-be63-965392f80877


Running optimization for KNN_k=20 with bm25...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0010476112365722656 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0021820068359375 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0010993480682373047 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0007810592651367188 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,bm25,20,0.250745,0.287349,0.300805,0.30381,0.328754,0.019425,"{'bm25_k1': 843.4301155557006, 'bm25_b': 0.763..."


In [9]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)


[I 2026-02-08 12:33:37,705] A new study created in memory with name: no-name-0c73b0b2-36b4-4360-b782-706cfeb4a222
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005199909210205078 seconds
  warnings.warn(
[I 2026-02-08 12:33:37,809] Trial 0 finished with value: 0.2346100478722425 and parameters: {}. Best is trial 0 with value: 0.2346100478722425.


Running optimization for KNN_k=20 with tfidf...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005776882171630859 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,tfidf,1,0.23461,0.26798,0.283676,0.286492,0.317622,0.023715,{}


In [10]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:38,139] A new study created in memory with name: no-name-88e85151-36ab-463c-a223-2e5c8f51a7a6
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005359649658203125 seconds
  warnings.warn(
[I 2026-02-08 12:33:38,234] Trial 0 finished with value: 0.23424453370163362 and parameters: {}. Best is trial 0 with value: 0.23424453370163362.


Running optimization for KNN_k=20 with log...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005905628204345703 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,log,1,0.234245,0.268971,0.283892,0.288141,0.317622,0.020064,{}


In [11]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="confidence", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:38,560] A new study created in memory with name: no-name-31dddd65-6956-496d-9f00-1e1941e94090


Running optimization for KNN_k=20 with confidence...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0008981227874755859 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0007750988006591797 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.001201629638671875 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0013327598571777344 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/si

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,confidence,20,0.234762,0.268589,0.283542,0.288141,0.317504,0.019218,{'conf_alpha': 55.13187607675897}


In [12]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:41,483] A new study created in memory with name: no-name-33e977c2-0f7a-45af-8cec-c360507a8773


Running optimization for KNN_k=20 with power...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0012192726135253906 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0010502338409423828 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0021445751190185547 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0012178421020507812 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/s

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,power,20,0.234443,0.268403,0.283895,0.287317,0.318096,0.019508,{'power_p': 0.507006817593925}


In [13]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:44,427] A new study created in memory with name: no-name-c3b44ff3-d9da-4471-a8e9-0ab59f8b7953
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005364418029785156 seconds
  warnings.warn(
[I 2026-02-08 12:33:44,523] Trial 0 finished with value: 0.24713364294316273 and parameters: {}. Best is trial 0 with value: 0.24713364294316273.


Running optimization for KNN_k=20 with normalized...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005817413330078125 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,normalized,1,0.247134,0.283596,0.298867,0.301666,0.331715,0.019569,{}


In [14]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:44,852] A new study created in memory with name: no-name-61dc4f7c-d549-4640-ac70-da3d07427978
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005140304565429688 seconds
  warnings.warn(
[I 2026-02-08 12:33:44,958] Trial 0 finished with value: 0.24875852135711207 and parameters: {}. Best is trial 0 with value: 0.24875852135711207.


Running optimization for KNN_k=20 with pmi...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005738735198974609 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,pmi,1,0.248759,0.284989,0.301401,0.304965,0.332662,0.019414,{}


In [15]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:45,290] A new study created in memory with name: no-name-caeeddc8-629b-4a54-9bad-af15db7f1d9a


Running optimization for KNN_k=20 with robust_user_centric...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0009341239929199219 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0020987987518310547 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.001873016357421875 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0011429786682128906 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/si

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,robust_user_centric,20,0.229796,0.270866,0.285502,0.289461,0.318806,0.019487,{'scale_factor': 2.6717703265899693}


In [16]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:48,241] A new study created in memory with name: no-name-bc1f240d-c8c1-4f8e-8352-dd274825721f


Running optimization for KNN_k=20 with robust_user_centric_weight_v2...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0011670589447021484 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0010957717895507812 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0007684230804443359 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0015799999237060547 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/s

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,robust_user_centric_weight_v2,20,0.235247,0.269902,0.284681,0.288471,0.318333,0.020085,"{'lower_q': 13.009637863954161, 'upper_q': 57...."


In [17]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:51,191] A new study created in memory with name: no-name-4a1bd885-d884-4f0b-b73a-c3804fb44ae7


Running optimization for KNN_k=20 with sigmoid_propensity...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0009493827819824219 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0009000301361083984 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0011093616485595703 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0012063980102539062 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/s

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,sigmoid_propensity,20,0.234007,0.269108,0.28341,0.288966,0.317622,0.019647,"{'p': 1.0751860580401422, 'beta': 0.0450192037..."


In [18]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="KNN_k=20", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:54,120] A new study created in memory with name: no-name-5963d933-a2e9-485c-905b-32bc9a3717a6


Running optimization for KNN_k=20 with power_lift...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0011663436889648438 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0014431476593017578 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0021812915802001953 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0020356178283691406 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/s

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=20,power_lift,20,0.246454,0.283755,0.29838,0.296388,0.324609,0.020371,{'p': 0.874576236478479}


In [19]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="no_weighting", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:57,053] A new study created in memory with name: no-name-c0dc67af-d273-46c3-bcd1-8ef8c84464a9
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005578994750976562 seconds
  warnings.warn(
[I 2026-02-08 12:33:57,185] Trial 0 finished with value: 0.21864221775713796 and parameters: {}. Best is trial 0 with value: 0.21864221775713796.


Running optimization for KNN_k=100 with no_weighting...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005810260772705078 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,no_weighting,1,0.218642,0.254108,0.265007,0.263071,0.288015,0.031892,{}


In [20]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="bm25", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:33:57,639] A new study created in memory with name: no-name-864451fe-40b4-4126-8fd0-edbf4a5be30c


Running optimization for KNN_k=100 with bm25...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0009665489196777344 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0012385845184326172 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0008189678192138672 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0014841556549072266 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/s

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,bm25,20,0.251625,0.293014,0.30985,0.30414,0.330886,0.028216,"{'bm25_k1': 338.7558204049336, 'bm25_b': 0.996..."


In [21]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="tfidf", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)


[I 2026-02-08 12:34:00,977] A new study created in memory with name: no-name-7e9750c3-2cc9-473f-bb2f-80262d6543ae
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005178451538085938 seconds
  warnings.warn(
[I 2026-02-08 12:34:01,111] Trial 0 finished with value: 0.2187007673634899 and parameters: {}. Best is trial 0 with value: 0.2187007673634899.


Running optimization for KNN_k=100 with tfidf...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005948543548583984 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,tfidf,1,0.218701,0.254346,0.265,0.262741,0.287305,0.042669,{}


In [22]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="log", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:34:01,578] A new study created in memory with name: no-name-972c0ac9-a1fe-4307-948d-a38f659bb65a
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005393028259277344 seconds
  warnings.warn(
[I 2026-02-08 12:34:01,714] Trial 0 finished with value: 0.2189509309995027 and parameters: {}. Best is trial 0 with value: 0.2189509309995027.


Running optimization for KNN_k=100 with log...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005948543548583984 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,log,1,0.218951,0.253354,0.265168,0.261917,0.288134,0.02852,{}


In [23]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="confidence", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:34:02,165] A new study created in memory with name: no-name-dc8d7ed8-0c10-4c00-9c42-0f7376e72130


Running optimization for KNN_k=100 with confidence...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.00116729736328125 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0024306774139404297 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0008606910705566406 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0013225078582763672 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/sit

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,confidence,20,0.219214,0.253918,0.265348,0.262246,0.288252,0.029165,{'conf_alpha': 17.627894967793516}


In [24]:
run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:34:05,505] A new study created in memory with name: no-name-7cf4b0bf-9b71-41f6-8962-08ad20fcadf0


Running optimization for KNN_k=100 with power...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0009784698486328125 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0013842582702636719 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0010972023010253906 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0010383129119873047 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/s

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,power,20,0.220329,0.254452,0.264948,0.263731,0.287778,0.04732,{'power_p': 0.6384267877860578}


In [25]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="normalized", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:34:09,039] A new study created in memory with name: no-name-1a3cac60-e103-467d-9d2c-f840a0bd4337
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005426406860351562 seconds
  warnings.warn(
[I 2026-02-08 12:34:09,171] Trial 0 finished with value: 0.2416550544240349 and parameters: {}. Best is trial 0 with value: 0.2416550544240349.


Running optimization for KNN_k=100 with normalized...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005893707275390625 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,normalized,1,0.241655,0.279204,0.295025,0.287152,0.31324,0.027931,{}


In [26]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="pmi", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:34:09,629] A new study created in memory with name: no-name-52e44623-6fdd-40c9-8de5-24214e5be53e
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005495548248291016 seconds
  warnings.warn(
[I 2026-02-08 12:34:09,763] Trial 0 finished with value: 0.2487289736907072 and parameters: {}. Best is trial 0 with value: 0.2487289736907072.


Running optimization for KNN_k=100 with pmi...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005936622619628906 seconds
  warnings.warn(


,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,pmi,1,0.248729,0.29483,0.309707,0.311397,0.335505,0.028297,{}


In [27]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:34:10,235] A new study created in memory with name: no-name-0edef15e-5ff6-4569-9fde-a87f7d49a468


Running optimization for KNN_k=100 with robust_user_centric...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0013332366943359375 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0008618831634521484 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0006356239318847656 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.001119375228881836 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/si

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,robust_user_centric,20,0.219065,0.255533,0.265849,0.263236,0.286594,0.031745,{'scale_factor': 4.261308278397727}


In [28]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="robust_user_centric_weight_v2", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:34:13,619] A new study created in memory with name: no-name-441df084-6a3d-46d5-9687-a3d5402936c7


Running optimization for KNN_k=100 with robust_user_centric_weight_v2...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0023162364959716797 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0008604526519775391 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0007412433624267578 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0010221004486083984 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/s

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,robust_user_centric_weight_v2,20,0.223226,0.25759,0.267439,0.264556,0.286949,0.047773,"{'lower_q': 11.265479498276028, 'upper_q': 56...."


In [29]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="sigmoid_propensity", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:34:17,110] A new study created in memory with name: no-name-dbc1e77f-c04f-4d8a-b173-bc5a0ecf543d


Running optimization for KNN_k=100 with sigmoid_propensity...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0008683204650878906 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0016307830810546875 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0014264583587646484 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0008327960968017578 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/s

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,sigmoid_propensity,20,0.219166,0.252285,0.261865,0.260762,0.281975,0.02931,"{'p': 0.2836829624877627, 'beta': 0.5821600768..."


In [30]:

run_hyperparameter_optimization(train_mat, val_mat, train_val_mat, test_mat, weighting_strategy="power_lift", algorithm="KNN_k=100", n_trials=20, output_dir=results_folder)

[I 2026-02-08 12:34:20,652] A new study created in memory with name: no-name-af9e18b3-8d1b-4b37-a0e7-f4c1561fd8d7


Running optimization for KNN_k=100 with power_lift...


/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0010879039764404297 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0023660659790039062 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0012269020080566406 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0013628005981445312 seconds
  warnings.warn(
/home/coder/.pyenv/versions/3.12.10/lib/python3.12/s

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,power_lift,20,0.252733,0.297673,0.314228,0.307439,0.33432,0.028892,{'p': 1.1627526935071715}


In [31]:
import glob

all_results = []
# Match any CSV in the result folder
for f in glob.glob(f"{results_folder}/*.csv"):
    all_results.append(pd.read_csv(f))

if all_results:
    experiment_results = pd.concat(all_results)
    experiment_results = experiment_results.sort_values("Test NDCG@20", ascending=False)
    experiment_results.to_csv(results_filename, index=False)
else:
    print("No results found.")

experiment_results

,Algorithm,Strategy,Number of Optimization Trials,Best Val NDCG@20,Test NDCG@10,Test NDCG@20,Test Precision@10,Test Precision@20,Final Train Time (s),Best Params
0,KNN_k=100,power_lift,20,0.252733,0.297673,0.314228,0.307439,0.334320,0.028892,{'p': 1.1627526935071715}
0,KNN_k=100,bm25,20,0.251625,0.293014,0.309850,0.304140,0.330886,0.028216,"{'bm25_k1': 338.7558204049336, 'bm25_b': 0.996..."
0,KNN_k=100,pmi,1,0.248729,0.294830,0.309707,0.311397,0.335505,0.028297,{}
0,KNN_k=20,pmi,1,0.248759,0.284989,0.301401,0.304965,0.332662,0.019414,{}
0,KNN_k=20,bm25,20,0.250745,0.287349,0.300805,0.303810,0.328754,0.019425,"{'bm25_k1': 843.4301155557006, 'bm25_b': 0.763..."
0,KNN_k=20,normalized,1,0.247134,0.283596,0.298867,0.301666,0.331715,0.019569,{}
0,KNN_k=20,power_lift,20,0.246454,0.283755,0.298380,0.296388,0.324609,0.020371,{'p': 0.874576236478479}
0,KNN_k=100,normalized,1,0.241655,0.279204,0.295025,0.287152,0.313240,0.027931,{}
0,KNN_k=20,robust_user_centric,20,0.229796,0.270866,0.285502,0.289461,0.318806,0.019487,{'scale_factor': 2.6717703265899693}
0,KNN_k=20,robust_user_centric_weight_v2,20,0.235247,0.269902,0.284681,0.288471,0.318333,0.020085,"{'lower_q': 13.009637863954161, 'upper_q': 57...."
